In [ ]:
!pip install mumin[all]==1.6.2 torchmetrics==0.7.2 --quiet
!pip install dgl-cu111==0.7.2 -f https://data.dgl.ai/wheels/repo.html --quiet

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

     |████████████████████████████████| 397 kB 3.3 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 211 kB 56.8 MB/s 
     |████████████████████████████████| 6.2 MB 57.9 MB/s 
     |████████████████████████████████| 4.7 MB 74.9 MB/s 
     |████████████████████████████████| 281 kB 84.0 MB/s 
     |████████████████████████████████| 93 kB 2.0 MB/s 
     |████████████████████████████████| 81 kB 9.5 MB/s 
     |████████████████████████████████| 7.4 MB 27.5 MB/s 
     |████████████████████████████████| 6.6 MB 43.6 MB/s 
     |████████████████████████████████| 120 kB 63.7 MB/s 
     |████████████████████████████████| 165.0 MB 30 kB/s 
Mon Sep 12 10:48:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-I

In [ ]:
import networkx as nx


import os
import random
from collections import defaultdict

import gensim
import networkx as nx
import numpy as np
import pkg_resources
from joblib import Parallel, delayed
from tqdm.auto import tqdm
from mumin import MuminDataset
import dgl
import torch



import random
from tqdm import tqdm

import torch
import argparse
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import torch
from torch.utils.data import Dataset
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import init

import os
from google.colab import drive
from getpass import getpass
from pathlib import Path
import shutil
import re
import random
import time


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


In [ ]:
twitter_bearer_token = 'AAAAAAAAAAAAAAAAAAAAAOYLagEAAAAA1K8YrEuA8CHQDAqAdjkPsBS2Pig%3DMUmnQgjpzkkXslyJpeNytAwFQ2qgiGE0Ah0rkrjuwH9UnOYSLI'
dataset = MuminDataset(twitter_bearer_token=twitter_bearer_token,size='small')
drive_dir = Path('drive')
drive.mount(str(drive_dir.resolve()), force_remount=True)
drive_content_dir = [child for child in drive_dir.iterdir() 
                     if re.search(r'My ?Drive', str(child.stem)) is not None][0]
shutil.copy(drive_content_dir / 'mumin-small.zip', 'mumin-small.zip')
dataset.compile()
dataset.add_embeddings()
dgl_g = dataset.to_dgl()

Mounted at /content/drive


INFO:mumin.dataset:Loading dataset
INFO:mumin.dataset:Outputting to DGL


In [ ]:

rel1 = ('user', 'posted', 'tweet')
rel2 = ('tweet', 'posted_inv', 'user')
rel3 = ('tweet', 'discusses', 'claim')
rel4 = ('claim', 'discusses_inv', 'tweet')
rel5 = ('user', 'follows', 'user')

sub_g = dgl.edge_type_subgraph(dgl_g,etypes=[rel1,rel2,rel3,rel4,rel5])


# sub_g = dgl.edge_type_subgraph(dgl_g,etypes=[rel3,rel4])

claim_id = []
for i in range(sub_g.num_nodes('claim')):
    claim_id.append('c'+str(i))
claim_id.__len__()

tweet_id = []
for i in range(sub_g.num_nodes('tweet')):
    tweet_id.append('t'+str(i))
tweet_id.__len__()

user_id = []
for i in range(sub_g.num_nodes('user')):
    user_id.append('u'+str(i))

tgt_node = 'claim'
dim = sub_g.nodes[tgt_node].data['feat'].shape[1]

In [ ]:
def generate_metapath(tgt_n,result_name,meta_path):
    # 拼接路径，输出路径
    import tqdm
    output_path = open(os.path.join(path, result_name), "w")
    count = 0

    # 看上一个cell实现的construct_graph()函数
    # hg: dgl里面定义的heterogeneous graph
    # author_names: author id: author name
    # hg, author_names, conf_names, paper_names = construct_graph()
    hg = sub_g


    # 关键：产生metapaths
    for conf_idx in tqdm.trange(hg.number_of_nodes(tgt_n)):
        # 产生"conference - paper - Author - paper - conference"类型的metapath
        traces, _ = dgl.sampling.random_walk(
            # 这里要改metapath！！！！！！！！！
                hg, [conf_idx] * num_walks_per_node, metapath=meta_path * walk_length)
        # 将paper都drop掉，只保留conference和Author
        # 注意conf_names和author_names这里使得输出的是name而不是id
        for tr in traces:
            outline = ' '.join(
                # 这里要改！！！！！！！！！！！！！！
                    claim_id[tr[i]]
                    for i in range(0, len(tr),5)
            )  # skip paper
            # outline = ' '
            # for i in range(0,len(tr)):
            #     if i%3 == 0:
            #         # print(tweet_id[tr[i]])
            #         outline = outline+' '+tweet_id[tr[i]]
            print(outline, file=output_path)
    output_path.close()


path = ''

# generate_metapath()

In [ ]:
class SkipGramModel(nn.Module):

    def __init__(self, emb_size, emb_dimension):
        super(SkipGramModel, self).__init__()
        self.emb_size = emb_size
        self.emb_dimension = emb_dimension
        self.u_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)
        self.v_embeddings = nn.Embedding(emb_size, emb_dimension, sparse=True)

        initrange = 1.0 / self.emb_dimension
        init.uniform_(self.u_embeddings.weight.data, -initrange, initrange)
        init.constant_(self.v_embeddings.weight.data, 0)

    def forward(self, pos_u, pos_v, neg_v):
        emb_u = self.u_embeddings(pos_u)
        emb_v = self.v_embeddings(pos_v)
        emb_neg_v = self.v_embeddings(neg_v)

        score = torch.sum(torch.mul(emb_u, emb_v), dim=1)
        score = torch.clamp(score, max=10, min=-10)
        score = -F.logsigmoid(score)

        neg_score = torch.bmm(emb_neg_v, emb_u.unsqueeze(2)).squeeze()
        neg_score = torch.clamp(neg_score, max=10, min=-10)
        neg_score = -torch.sum(F.logsigmoid(-neg_score), dim=1)

        return torch.mean(score + neg_score)

    def save_embedding(self, id2word, file_name):
        embedding = self.u_embeddings.weight.cpu().data.numpy()
        with open(file_name, 'w') as f:
            f.write('%d %d\n' % (len(id2word), self.emb_dimension))
            for wid, w in id2word.items():
                e = ' '.join(map(lambda x: str(x), embedding[wid]))
                f.write('%s %s\n' % (w, e))


class CustomDataset(object):
    """
    Custom dataset generated by sampler.py (e.g. NetDBIS)
    """
    def __init__(self, path):
        self.fn = path

class DataReader:
    NEGATIVE_TABLE_SIZE = 1e8

    def __init__(self, dataset, min_count, care_type):

        self.negatives = []
        self.discards = []
        self.negpos = 0
        self.care_type = care_type
        self.word2id = dict()
        self.id2word = dict()
        self.sentences_count = 0
        self.token_count = 0
        self.word_frequency = dict()
        self.inputFileName = dataset.fn
        self.read_words(min_count)
        self.initTableNegatives()
        self.initTableDiscards()

    def read_words(self, min_count):
        word_frequency = dict()
        for line in open(self.inputFileName, encoding="ISO-8859-1"):
            line = line.split()
            if len(line) > 1:
                self.sentences_count += 1
                for word in line:
                    if len(word) > 0:
                        self.token_count += 1
                        word_frequency[word] = word_frequency.get(word, 0) + 1

                        if self.token_count % 1000000 == 0:
                            print("Read " + str(int(self.token_count / 1000000)) + "M words.")

        wid = 0
        for w, c in word_frequency.items():
            if c < min_count:
                continue
            self.word2id[w] = wid
            self.id2word[wid] = w
            self.word_frequency[wid] = c
            wid += 1

        self.word_count = len(self.word2id)
        print("Total embeddings: " + str(len(self.word2id)))

    def initTableDiscards(self):
        # get a frequency table for sub-sampling. Note that the frequency is adjusted by
        # sub-sampling tricks.
        t = 0.0001
        f = np.array(list(self.word_frequency.values())) / self.token_count
        self.discards = np.sqrt(t / f) + (t / f)

    def initTableNegatives(self):
        # get a table for negative sampling, if word with index 2 appears twice, then 2 will be listed
        # in the table twice.
        pow_frequency = np.array(list(self.word_frequency.values())) ** 0.75
        words_pow = sum(pow_frequency)
        ratio = pow_frequency / words_pow
        count = np.round(ratio * DataReader.NEGATIVE_TABLE_SIZE)
        for wid, c in enumerate(count):
            self.negatives += [wid] * int(c)
        self.negatives = np.array(self.negatives)
        np.random.shuffle(self.negatives)
        self.sampling_prob = ratio

    def getNegatives(self, target, size):  # TODO check equality with target
        if self.care_type == 0:
            response = self.negatives[self.negpos:self.negpos + size]
            self.negpos = (self.negpos + size) % len(self.negatives)
            if len(response) != size:
                return np.concatenate((response, self.negatives[0:self.negpos]))
        return response


# -----------------------------------------------------------------------------------------------------------------

class Metapath2vecDataset(Dataset):
    def __init__(self, data, window_size):
        # read in data, window_size and input filename
        self.data = data
        self.window_size = window_size
        self.input_file = open(data.inputFileName, encoding="ISO-8859-1")

    def __len__(self):
        # return the number of walks
        return self.data.sentences_count

    def __getitem__(self, idx):
        # return the list of pairs (center, context, 5 negatives)
        while True:
            line = self.input_file.readline()
            if not line:
                self.input_file.seek(0, 0)
                line = self.input_file.readline()

            if len(line) > 1:
                words = line.split()

                if len(words) > 1:
                    word_ids = [self.data.word2id[w] for w in words if
                                w in self.data.word2id and np.random.rand() < self.data.discards[self.data.word2id[w]]]

                    pair_catch = []
                    for i, u in enumerate(word_ids):
                        for j, v in enumerate(
                                word_ids[max(i - self.window_size, 0):i + self.window_size]):
                            assert u < self.data.word_count
                            assert v < self.data.word_count
                            if i == j:
                                continue
                            pair_catch.append((u, v, self.data.getNegatives(v,5)))
                    return pair_catch


    @staticmethod
    def collate(batches):
        all_u = [u for batch in batches for u, _, _ in batch if len(batch) > 0]
        all_v = [v for batch in batches for _, v, _ in batch if len(batch) > 0]
        all_neg_v = [neg_v for batch in batches for _, _, neg_v in batch if len(batch) > 0]

        return torch.LongTensor(np.array(all_u)), torch.LongTensor(np.array(all_v)), torch.LongTensor(np.array(all_neg_v))

class Metapath2VecTrainer:
    def __init__(self, args):
        if args.aminer:
            dataset = AminerDataset(args.path)
        else:
            # 我们这里使用这种情况
            # 指定path: NetDBIS
            dataset = CustomDataset(args.path)
        # 读入metapath文件并处理
        self.data = DataReader(dataset, args.min_count, args.care_type)
        # 实现封装
        dataset = Metapath2vecDataset(self.data, args.window_size)
        # https://zhuanlan.zhihu.com/p/30385675
        self.dataloader = DataLoader(dataset, batch_size=args.batch_size,
                                     shuffle=True, num_workers=args.num_workers, collate_fn=dataset.collate)
        
        # 模型参数用args赋值
        self.output_file_name = args.output_file
        # 一共多少个节点N
        self.emb_size = len(self.data.word2id)
        self.emb_dimension = args.dim
        self.batch_size = args.batch_size
        self.iterations = args.iterations
        self.initial_lr = args.initial_lr
        # emb_size: 一共多少个节点N; emb_dimension: 维度如128
        self.skip_gram_model = SkipGramModel(self.emb_size, self.emb_dimension)

        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")
        if self.use_cuda:
            self.skip_gram_model.cuda()

    def train(self):

        for iteration in range(self.iterations):
            # print("\n\n\nIteration: " + str(iteration + 1))
            # 优化方式
            optimizer = optim.SparseAdam(self.skip_gram_model.parameters(), lr=self.initial_lr)
            # 优化方式和学习率等
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(self.dataloader))

            running_loss = 0.0
            # 按batch训练
            for i, sample_batched in enumerate(tqdm(self.dataloader)):

                if len(sample_batched[0]) > 1:
                    pos_u = sample_batched[0].to(self.device)
                    pos_v = sample_batched[1].to(self.device)
                    neg_v = sample_batched[2].to(self.device)

                    
                    optimizer.zero_grad()
                    # 损失函数值
                    loss = self.skip_gram_model.forward(pos_u, pos_v, neg_v)
                    # 反向传播，更新参数
                    loss.backward()
                    optimizer.step()
                    #有改动@@@@@@@@@@@@@@@@@@@@@@@
                    scheduler.step()

                    running_loss = running_loss * 0.9 + loss.item() * 0.1
                    # if i > 0 and i % 500 == 0:
                    #     print(" Loss: " + str(running_loss))

            self.skip_gram_model.save_embedding(self.data.id2word, self.output_file_name)

In [ ]:
def getData():
    feats = sub_g.nodes[tgt_node].data['feat'].cpu().detach().numpy()
    text_feats= []
    for i in range(feats.shape[0]):
        text_feats.append(feats[i,:])

    label = sub_g.nodes[tgt_node].data['label'].cpu().detach().numpy()
    train_mask = sub_g.nodes[tgt_node].data['train_mask'].cpu().detach().numpy()
    val_mask = sub_g.nodes[tgt_node].data['val_mask'].cpu().detach().numpy()
    test_mask = sub_g.nodes[tgt_node].data['test_mask'].cpu().detach().numpy()

    import pandas as pd
    dataframe = pd.DataFrame(label,columns=['label'])
    dataframe['train_mask'] = train_mask.tolist()
    dataframe['val_mask'] = val_mask.tolist()
    dataframe['test_mask'] = test_mask.tolist()
    dataframe['text_feats'] = text_feats

    feats_list = []
    id_list = []
    with open('result_'+name, "r") as f:
        for line in f.readlines():
            line = line.strip('\n')
            #去掉列表中每一个元素的换行符
            feats = []
            for idx,word in enumerate(line.split()):
                if idx == 0:
                    id_list.append(word)
                else:
                    feats.append(word)
            feats_list.append(np.array(feats,dtype=float))

    feat_df = pd.DataFrame({'feats':feats_list})
    feat_df['id'] = id_list
    feat_df = feat_df.drop(index=[0])

    mask = [ True if i.startswith('c') else False for i in feat_df.id ]
    feat_df = feat_df[mask]

    num_id = [int(word[1:]) for word in feat_df.id]
    feat_df['num_id'] = num_id

    feat_df = feat_df.sort_values(by = 'num_id')
    feat_df.reset_index(drop=True, inplace=True)
    tweet_claim_df = dataframe.join(feat_df)
    tweet_claim_df = tweet_claim_df.dropna()

    mix_feats = []
    for i in range(len(tweet_claim_df)):
        mix_feats.append(np.hstack((tweet_claim_df['feats'][i],tweet_claim_df['text_feats'][i])))
    tweet_claim_df['mix_feats'] = mix_feats

    from sklearn.model_selection import train_test_split
    # Split up the data
    train = tweet_claim_df.query('train_mask == True')
    val = tweet_claim_df.query('val_mask == True')
    test = tweet_claim_df.query('test_mask == True')

    train.reset_index(drop=True, inplace=True)
    val.reset_index(drop=True, inplace=True)
    test.reset_index(drop=True, inplace=True)
    return train,val,test

In [ ]:
def getFeatsMatrix(df):
    m_temp = np.zeros([len(df['feats'].values),dim])
    for i in range(len(df['feats'].values)):
        m_temp[i,:] = df['feats'][i]
    return m_temp

def getTextFeatsMatrix(df):
    m_temp = np.zeros([len(df['text_feats'].values),dim])
    for i in range(len(df['text_feats'].values)):
        m_temp[i,:] = df['text_feats'][i]
    return m_temp

def getMixFeatsMatrix(df):
    m_temp = np.zeros([len(df['mix_feats'].values),2*dim])
    for i in range(len(df['mix_feats'].values)):
        m_temp[i,:] = df['mix_feats'][i]
    return m_temp

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
def convert_to_data_loader(dataset,m, num_classes):
    # convert from list to tensor
    input_tensor = torch.from_numpy(m).float()
    label_tensor = torch.from_numpy(np.array(dataset['label'])).long()
    tensor_dataset = TensorDataset(input_tensor, label_tensor)
    loader = DataLoader(tensor_dataset, batch_size=64, shuffle=True)

    return loader

num_classes = 2   # number of possible labels in the sentiment analysis task

In [ ]:
from torch import nn, optim
import torch.nn.functional as F


class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 2)


        # 构造Dropout方法，在每次训练过程中都随机“掐死”百分之二十的神经元，防止过拟合。
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        # 确保输入的tensor是展开的单列数据，把每张图片的通道、长度、宽度三个维度都压缩为一列
        x = x.view(x.shape[0], -1)

        # 在训练过程中对隐含层神经元的正向推断使用Dropout方法
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))

        # 在输出单元不需要使用Dropout方法
        x = F.log_softmax(self.fc4(x), dim=1)

        return x

In [ ]:
def train_nn():
    # 对上面定义的Classifier类进行实例化
    model = Classifier()

    # 定义损失函数为负对数损失函数
    criterion = nn.NLLLoss()

    # 优化方法为Adam梯度下降方法，学习率为0.003
    optimizer = optim.Adam(model.parameters(), lr=0.003)

    # 对训练集的全部数据学习15遍，这个数字越大，训练时间越长
    epochs = 15

    # 将每次训练的训练误差和测试误差存储在这两个列表里，后面绘制误差变化折线图用
    train_losses, test_losses = [], []

    for e in range(epochs):
        running_loss = 0

        # 对训练集中的所有图片都过一遍
        for images, labels in train_loader:
            # 将优化器中的求导结果都设为0，否则会在每次反向传播之后叠加之前的
            optimizer.zero_grad()

            # 对64张图片进行推断，计算损失函数，反向传播优化权重，将损失求和
            log_ps = model(images)
            loss = criterion(log_ps, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # 每次学完一遍数据集，都进行以下测试操作
        else:
            test_loss = 0
            accuracy = 0
            # 测试的时候不需要开自动求导和反向传播
            with torch.no_grad():
                # 关闭Dropout
                model.eval()

                # 对测试集中的所有图片都过一遍
                for images, labels in dev_loader:
                    # 对传入的测试集图片进行正向推断、计算损失，accuracy为测试集一万张图片中模型预测正确率
                    log_ps = model(images)
                    test_loss += criterion(log_ps, labels)
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)

                    # 等号右边为每一批64张测试图片中预测正确的占比
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
            # 恢复Dropout
            model.train()
            # 将训练误差和测试误差存在两个列表里，后面绘制误差变化折线图用
            train_losses.append(running_loss/len(train_loader))
            test_losses.append(test_loss/len(dev_loader))


    return model

In [ ]:
def predict_nn(trained_model, test_loader):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
    model.eval()

    correct = 0  # count the number of correct classification labels

    gold_labs = []  # gold labels to return
    pred_labs = []  # predicted labels to return

    for inputs, labels in test_loader:
        test_output = trained_model(inputs)
        predicted_labels = test_output.argmax(1)

        gold_labs.extend(labels.tolist())
        pred_labs.extend(predicted_labels.tolist())



    f1 = f1_score(gold_labs, pred_labs, average='macro')
    return f1

In [ ]:
f1_wl = []
for wl in range(5,106,10):
    num_walks_per_node = 50
    # random walk的长度
    walk_length = wl
    metapath = ['discusses_inv','posted_inv','follows','posted','discusses']
    name = 'ctuutc'+str(num_walks_per_node)+'_'+str(walk_length)+'.txt'
    generate_metapath(tgt_node,name,metapath)

    parser = argparse.ArgumentParser(description="Metapath2vec")
    #parser.add_argument('--input_file', type=str, help="input_file")
    parser.add_argument('--aminer', action='store_true', help='Use AMiner dataset')
    # 输入文件
    parser.add_argument('--path', type=str, help="input_path", default=name)
    # 输出文件
    parser.add_argument('--output_file', type=str, help='output_file', default='result_'+name)
    # embedding维度
    parser.add_argument('--dim', default=dim, type=int, help="embedding dimensions")
    # 窗口大小
    parser.add_argument('--window_size', default=7, type=int, help="context window size")
    # 迭代次数
    parser.add_argument('--iterations', default=5, type=int, help="iterations")
    # batch size
    parser.add_argument('--batch_size', default=50, type=int, help="batch size")
    # 0: metapath2vec; 1: metapath2vec++
    parser.add_argument('--care_type', default=0, type=int, help="if 1, heterogeneous negative sampling, else normal negative sampling")
    # 学习率
    parser.add_argument('--initial_lr', default=0.025, type=float, help="learning rate")
    # skip2gram模型的词频
    parser.add_argument('--min_count', default=5, type=int, help="min count")
    # 资源核数
    parser.add_argument('--num_workers', default=16, type=int, help="number of workers")
    args = parser.parse_args(args=[])
    from tqdm import tqdm
    m2v = Metapath2VecTrainer(args)
    m2v.train()

    train,val,test = getData()

    train_loader = convert_to_data_loader(train,getFeatsMatrix(train), num_classes)
    dev_loader = convert_to_data_loader(val, getFeatsMatrix(val),num_classes)
    test_loader = convert_to_data_loader(test,getFeatsMatrix(test), num_classes)

    score = []

    for i in range(20):
        model = train_nn()
        score.append(predict_nn(model,test_loader))

    print(f'F1 score (macro average) = {sorted(score)[-1]}')
    f1_wl.append(sorted(score)[-1])

In [ ]:
f1_wl

[0.6565920273119792,
 0.5476190476190477,
 0.625615763546798,
 0.5777777777777777,
 0.5650165679285069,
 0.7223296797764883,
 0.592128801431127,
 0.5966981132075472,
 0.507899507899508,
 0.626003626003626,
 0.5396915125575386]

In [ ]:
f1_nw = []
for nw in range(20,201,20):
    num_walks_per_node = nw
    # random walk的长度
    walk_length = 50
    metapath = ['discusses_inv','posted_inv','follows','posted','discusses']
    name = 'ctuutc'+str(num_walks_per_node)+'_'+str(walk_length)+'.txt'
    generate_metapath(tgt_node,name,metapath)

    parser = argparse.ArgumentParser(description="Metapath2vec")
    #parser.add_argument('--input_file', type=str, help="input_file")
    parser.add_argument('--aminer', action='store_true', help='Use AMiner dataset')
    # 输入文件
    parser.add_argument('--path', type=str, help="input_path", default=name)
    # 输出文件
    parser.add_argument('--output_file', type=str, help='output_file', default='result_'+name)
    # embedding维度
    parser.add_argument('--dim', default=dim, type=int, help="embedding dimensions")
    # 窗口大小
    parser.add_argument('--window_size', default=7, type=int, help="context window size")
    # 迭代次数
    parser.add_argument('--iterations', default=5, type=int, help="iterations")
    # batch size
    parser.add_argument('--batch_size', default=50, type=int, help="batch size")
    # 0: metapath2vec; 1: metapath2vec++
    parser.add_argument('--care_type', default=0, type=int, help="if 1, heterogeneous negative sampling, else normal negative sampling")
    # 学习率
    parser.add_argument('--initial_lr', default=0.025, type=float, help="learning rate")
    # skip2gram模型的词频
    parser.add_argument('--min_count', default=5, type=int, help="min count")
    # 资源核数
    parser.add_argument('--num_workers', default=16, type=int, help="number of workers")
    args = parser.parse_args(args=[])
    from tqdm import tqdm
    m2v = Metapath2VecTrainer(args)
    m2v.train()

    train,val,test = getData()

    train_loader = convert_to_data_loader(train,getFeatsMatrix(train), num_classes)
    dev_loader = convert_to_data_loader(val, getFeatsMatrix(val),num_classes)
    test_loader = convert_to_data_loader(test,getFeatsMatrix(test), num_classes)

    score = []

    for i in range(20):
        model = train_nn()
        score.append(predict_nn(model,test_loader))

    print(f'F1 score (macro average) = {sorted(score)[-1]}, num_walks = {nw}')
    f1_nw.append(sorted(score)[-1])

In [ ]:
f1_nw1 = []
for nw in range(200,501,100):
    num_walks_per_node = nw
    # random walk的长度
    walk_length = 50
    metapath = ['discusses_inv','posted_inv','follows','posted','discusses']
    name = 'ctuutc'+str(num_walks_per_node)+'_'+str(walk_length)+'.txt'
    generate_metapath(tgt_node,name,metapath)

    parser = argparse.ArgumentParser(description="Metapath2vec")
    #parser.add_argument('--input_file', type=str, help="input_file")
    parser.add_argument('--aminer', action='store_true', help='Use AMiner dataset')
    # 输入文件
    parser.add_argument('--path', type=str, help="input_path", default=name)
    # 输出文件
    parser.add_argument('--output_file', type=str, help='output_file', default='result_'+name)
    # embedding维度
    parser.add_argument('--dim', default=dim, type=int, help="embedding dimensions")
    # 窗口大小
    parser.add_argument('--window_size', default=7, type=int, help="context window size")
    # 迭代次数
    parser.add_argument('--iterations', default=5, type=int, help="iterations")
    # batch size
    parser.add_argument('--batch_size', default=50, type=int, help="batch size")
    # 0: metapath2vec; 1: metapath2vec++
    parser.add_argument('--care_type', default=0, type=int, help="if 1, heterogeneous negative sampling, else normal negative sampling")
    # 学习率
    parser.add_argument('--initial_lr', default=0.025, type=float, help="learning rate")
    # skip2gram模型的词频
    parser.add_argument('--min_count', default=5, type=int, help="min count")
    # 资源核数
    parser.add_argument('--num_workers', default=16, type=int, help="number of workers")
    args = parser.parse_args(args=[])
    from tqdm import tqdm
    m2v = Metapath2VecTrainer(args)
    m2v.train()

    train,val,test = getData()

    train_loader = convert_to_data_loader(train,getFeatsMatrix(train), num_classes)
    dev_loader = convert_to_data_loader(val, getFeatsMatrix(val),num_classes)
    test_loader = convert_to_data_loader(test,getFeatsMatrix(test), num_classes)

    model = train_nn()
    score = predict_nn(model,test_loader)
    print(f'F1 score (macro average) = {score}')

    print(f'F1 score (macro average) = {score}, num_walks = {nw}')
    f1_nw1.append(score)

100%|██████████| 2100/2100 [00:53<00:00, 38.96it/s]


Read 1M words.
Read 2M words.
Read 3M words.
Read 4M words.
Read 5M words.
Read 6M words.
Read 7M words.
Read 8M words.
Read 9M words.
Read 10M words.
Read 11M words.
Read 12M words.
Read 13M words.
Read 14M words.
Read 15M words.
Read 16M words.
Read 17M words.
Read 18M words.
Read 19M words.
Read 20M words.
Read 21M words.
Total embeddings: 2100


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 8400/8400 [00:44<00:00, 187.95it/s]


F1 score (macro average) = 0.6431924882629108, num_walks = 200


100%|██████████| 2100/2100 [01:13<00:00, 28.71it/s]


...

F1 score (macro average) = 0.617930456640134, num_walks = 400


100%|██████████| 2100/2100 [01:59<00:00, 17.62it/s]


Read 1M words.
Read 2M words.
Read 3M words.
Read 4M words.
Read 5M words.
Read 6M words.
Read 7M words.
Read 8M words.
Read 9M words.
Read 10M words.
Read 11M words.
Read 12M words.
Read 13M words.
Read 14M words.
Read 15M words.
Read 16M words.
Read 17M words.
Read 18M words.
Read 19M words.
Read 20M words.
Read 21M words.
Read 22M words.
Read 23M words.
Read 24M words.
Read 25M words.
Read 26M words.
Read 27M words.
Read 28M words.
Read 29M words.
Read 30M words.
Read 31M words.
Read 32M words.
Read 33M words.
Read 34M words.
Read 35M words.
Read 36M words.
Read 37M words.
Read 38M words.
Read 39M words.
Read 40M words.
Read 41M words.
Read 42M words.
Read 43M words.
Read 44M words.
Read 45M words.
Read 46M words.
Read 47M words.
Read 48M words.
Read 49M words.
Read 50M words.
Read 51M words.
Read 52M words.
Read 53M words.
Total embeddings: 2100


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 21000/21000 [01:48<00:00, 193.87it/s]


F1 score (macro average) = 0.625615763546798, num_walks = 500


In [ ]:
f1_nw1

[0.6431924882629108, 0.6755336617405583, 0.617930456640134, 0.625615763546798]

In [ ]:
f1_wl = []
for wl in range(55,56):
    num_walks_per_node = 50
    # random walk的长度
    walk_length = wl
    metapath = ['discusses_inv','posted_inv','follows','posted','discusses']
    name = 'ctuutc'+str(num_walks_per_node)+'_'+str(walk_length)+'.txt'
    generate_metapath(tgt_node,name,metapath)

    parser = argparse.ArgumentParser(description="Metapath2vec")
    #parser.add_argument('--input_file', type=str, help="input_file")
    parser.add_argument('--aminer', action='store_true', help='Use AMiner dataset')
    # 输入文件
    parser.add_argument('--path', type=str, help="input_path", default=name)
    # 输出文件
    parser.add_argument('--output_file', type=str, help='output_file', default='result_'+name)
    # embedding维度
    parser.add_argument('--dim', default=dim, type=int, help="embedding dimensions")
    # 窗口大小
    parser.add_argument('--window_size', default=7, type=int, help="context window size")
    # 迭代次数
    parser.add_argument('--iterations', default=5, type=int, help="iterations")
    # batch size
    parser.add_argument('--batch_size', default=50, type=int, help="batch size")
    # 0: metapath2vec; 1: metapath2vec++
    parser.add_argument('--care_type', default=0, type=int, help="if 1, heterogeneous negative sampling, else normal negative sampling")
    # 学习率
    parser.add_argument('--initial_lr', default=0.025, type=float, help="learning rate")
    # skip2gram模型的词频
    parser.add_argument('--min_count', default=5, type=int, help="min count")
    # 资源核数
    parser.add_argument('--num_workers', default=16, type=int, help="number of workers")
    args = parser.parse_args(args=[])
    from tqdm import tqdm
    m2v = Metapath2VecTrainer(args)
    m2v.train()

    train,val,test = getData()

    train_loader = convert_to_data_loader(train,getFeatsMatrix(train), num_classes)
    dev_loader = convert_to_data_loader(val, getFeatsMatrix(val),num_classes)
    test_loader = convert_to_data_loader(test,getFeatsMatrix(test), num_classes)

    model = train_nn()
    score = predict_nn(model,test_loader)
    print(f'F1 score (macro average) = {score}')

    print(f'F1 score (macro average) = {score}, num_walks = {nw}')
    f1_nw1.append(score)

100%|██████████| 2100/2100 [00:15<00:00, 137.40it/s]


Read 1M words.
Read 2M words.
Read 3M words.
Read 4M words.
Read 5M words.
Total embeddings: 2100


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100%|██████████| 2100/2100 [00:13<00:00, 154.18it/s]


F1 score (macro average) = 0.6402784222737818


In [ ]:
f1_nw

[0.5866355866355867,
 0.5465909090909091,
 0.5615384615384615,
 0.625615763546798,
 0.5756978653530377,
 0.643020594965675,
 0.602840680691411,
 0.5765161915529355,
 0.592128801431127,
 0.657543391188251]